In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib widget
from collections import defaultdict
import geopandas as gpd
import textwrap
import os
import glob
from shapely.geometry import Polygon

pulje_størrelse = 7
max_teams_per_club = 3
outlier_factor = 1.4
fairness_factor = 0.5

## Data

### Udregn køretid mellem klubber 

In [2]:
def computeGEOtable(geo_data, server="127.0.0.1:5000", profile="car", annotations="duration,distance"): 
    # Create the coordinates list for sources and destinations
    address_coords = geo_data[['Hjemmebane lng', 'Hjemmebane lat']].apply(lambda row: f"{row['Hjemmebane lng']},{row['Hjemmebane lat']}", axis=1).tolist()
    coordinates_str = ";".join(address_coords)
    num_clubs = len(address_coords)
    sources_str = ";".join(map(str, range(num_clubs)))
    destinations_str = sources_str

    local_url = f"http://{server}/table/v1/{profile}/{coordinates_str}?sources={sources_str}&destinations={destinations_str}&annotations={annotations}&exclude=ferry"

    try:
        # Make the API request
        response = requests.get(local_url)
        response.raise_for_status()
        table_data = response.json()

        results = pd.DataFrame(columns=["Hjemmebane", "Udebane", "Køretid", "Afstand"])
        
        for i in tqdm(range(num_clubs)):
            for j in range(i, num_clubs): 
                if table_data['durations'][i][j] is not None:
                    duration = table_data['durations'][i][j] / 60 / 60  # hours
                    distance = table_data['distances'][i][j] / 1000      # km

                    source = geo_data["Hjemmebane"][i]
                    destination = geo_data["Hjemmebane"][j]
                    results.loc[len(results)] = [source, destination, duration, distance]
                    results.loc[len(results)] = [destination, source, duration, distance]
                    
    except requests.exceptions.RequestException as e:
        print(f"Failed to connect to OSRM server: {e}")

    return results

In [3]:
def forbered_data(path):
    # Indlæs data med hold_df
    hold_df = pd.read_excel(path)
    
    hold_df.loc[:,"Hjemmebane lng"] = hold_df.loc[:,"Hjemmebane lng"] + np.random.uniform(-0.05, 0.05, len(hold_df))
    hold_df.loc[:,"Hjemmebane lat"] = hold_df.loc[:,"Hjemmebane lat"] + np.random.uniform(-0.05, 0.05, len(hold_df))

    hold_df.loc[:,"Division"] = hold_df.loc[:, "Division"].str.replace(" - A", "", regex=False)
    hold_df.loc[:,"Division"] = hold_df.loc[:, "Division"].str.replace(" - B", "", regex=False)
    klubber_df = hold_df.drop_duplicates(subset="Hjemmebane").reset_index(drop=True)

    klubber_df = klubber_df[~klubber_df["Hjemmebane lat"].isna()]
    klubber_df = klubber_df.reset_index()
        
    # køretider_df = computeGEOtable(klubber_df)
    køretider_df = pd.read_csv("data/proc/køretider.csv")
    køretider_df.to_csv("data/proc/køretider.csv", index=False)
    
    return hold_df, klubber_df, køretider_df

### Beregning af køretider

In [4]:
def beregn_alle_køretider_i_række(række_df):
    """Beregner alle køretider i række"""
    alle_køretider_i_række = række_df[["Team id","Hjemmebane"]].merge(række_df[["Team id","Hjemmebane"]], how='cross')
    alle_køretider_i_række = alle_køretider_i_række.merge(køretider, left_on=["Hjemmebane_x", "Hjemmebane_y"], right_on=["Hjemmebane", "Udebane"], how='left', suffixes=('_left', '_right'))
    alle_køretider_i_række = alle_køretider_i_række.drop(columns=["Hjemmebane_x", "Hjemmebane_y"])
    alle_køretider_i_række = alle_køretider_i_række.rename(columns={"Team id_x": "Team id hjemmebane", "Team id_y": "Team id udebane"})
    alle_køretider_i_række = alle_køretider_i_række.drop_duplicates(subset=["Team id hjemmebane", "Team id udebane"])
    
    distance = defaultdict(int)

    for _, row in alle_køretider_i_række.iterrows():
        i = row["Team id hjemmebane"]
        j = row["Team id udebane"]
        distance[(i, j)] += row["Køretid"]
        distance[(j, i)] += row["Køretid"]
    
    return alle_køretider_i_række, distance

## Ny grådig

In [5]:
def ligeligt_fordelte_puljer(række_df,pulje_størrelse):
    coords = række_df[["Hjemmebane lng", "Hjemmebane lat"]].to_numpy()
    team_ids = række_df["Team id"].to_numpy()
    def round_to_nearest(x):
        possible_vals = [1,2,4,8,16]
        return possible_vals[np.argmin(np.abs(np.array(possible_vals)-x))]
        
    k = round_to_nearest(len(række_df)/pulje_størrelse)
    n = len(coords)

    def dist(a, b):
        return np.linalg.norm(a - b)

    # deterministic start: most central team
    center = coords.mean(axis=0)
    start_idx = np.argmin(np.linalg.norm(coords - center, axis=1))

    selected = [start_idx]
    distances = np.array([dist(coords[start_idx], coords[i]) for i in range(n)])

    for _ in range(1, k):
        next_idx = np.argmax(distances)
        selected.append(next_idx)
        for i in range(n):
            distances[i] = min(distances[i], dist(coords[next_idx], coords[i]))

    return team_ids[selected]

In [6]:
def grådig_fordeling(række_df):
    alle_hold = set(række_df["Team id"])
    seeds = ligeligt_fordelte_puljer(række_df, pulje_størrelse)
    puljer = [[seed] for seed in seeds]
    ledige_hold = alle_hold - set(seeds)

    while ledige_hold:
        for pulje in puljer:
            clubs_in_pulje = {hold_til_klub[t] for t in pulje}
            candidates = []

            neigh = alle_køretider_i_række[
                (alle_køretider_i_række["Team id hjemmebane"] == pulje[0]) &
                (alle_køretider_i_række["Team id udebane"].isin(ledige_hold)) &
                (~alle_køretider_i_række["Udebane"].isin(clubs_in_pulje))
            ]

            if not neigh.empty:
                best = neigh.sort_values("Køretid").iloc[0]
                candidates.append((best["Køretid"], best["Team id udebane"]))

            if candidates:
                chosen = min(candidates)[1]
            else:
                fallback = [h for h in ledige_hold if hold_til_klub[h] not in clubs_in_pulje]
                if not fallback:
                    fallback = [h for h in ledige_hold]
                    if len(fallback)==0:
                        break
                    
                chosen = fallback[0]

            pulje.append(chosen)
            ledige_hold.remove(chosen)

    return puljer


### Improvement algo helper functions

In [7]:
# ======================================================
# Distance utilities
# ======================================================

def make_distance_dict(alle_køretider_i_række):
    distance = defaultdict(float)
    for _, row in alle_køretider_i_række.iterrows():
        i = row["Team id hjemmebane"]
        j = row["Team id udebane"]
        distance[(i, j)] += row["Køretid"]
        distance[(j, i)] += row["Køretid"]
    return distance


# ======================================================
#  distance metrics
# ======================================================

def avg_team_distance(team, pulje, distance):
    return sum(distance[(team, t)] for t in pulje if t != team) / (len(pulje) - 1)

def max_team_distance(team, pulje, distance):
    return max(distance[(team, t)] for t in pulje if t != team)

def avg_pulje_distance(pulje, distance):
    return np.mean([avg_team_distance(t, pulje, distance) for t in pulje])

def max_avg_pulje_distance(pulje, distance):
    return max(avg_team_distance(t, pulje, distance) for t in pulje)

def avg_fordeling_distance(puljer, distance):
    return np.mean([avg_pulje_distance(p, distance) for p in puljer])

def max_fordeling_distance(puljer, distance):
    return max(max_avg_pulje_distance(p, distance) for p in puljer)


# ======================================================
#  outlier definitions
# ======================================================

def is_outlier(team, pulje, distance, outlier_factor):
    team_val = avg_team_distance(team, pulje, distance)
    others = [avg_team_distance(t, pulje, distance) for t in pulje if t != team]
    return team_val > outlier_factor * np.mean(others)


def pulje_is_outlier_idx(idx, puljer, distance, outlier_factor):
    this = avg_pulje_distance(puljer[idx], distance)
    others = [avg_pulje_distance(p, distance) for i, p in enumerate(puljer) if i != idx]
    return this > outlier_factor * np.mean(others)

def driving_time_exceeds_cap(team, pulje, distance, cap):
    team_times = [distance[(team, t)] for t in pulje if t != team]
    return any(time > cap for time in team_times)

# def average_pulje_driving_time_is_over_cap(pulje, distance, cap):
#     team_avg_times = [avg_team_distance(t, pulje, distance) for t in pulje]
#     avg_time = np.mean(team_avg_times)
#     return avg_time > cap


# ======================================================
#  check constraints
# ======================================================

def club_count_ok(pulje, hold_til_klub, max_teams_per_club):
    counts = {}
    for t in pulje:
        c = hold_til_klub[t]
        counts[c] = counts.get(c, 0) + 1
        if counts[c] > max_teams_per_club:
            return False
    return True


def club_constraint_ok(old_a, old_b, new_a, new_b, i, j, a, b, puljer, distance, hold_til_klub,outlier_factor, max_teams_per_club):
    # Hard cap
    if not (club_count_ok(new_a, hold_til_klub, max_teams_per_club) and
            club_count_ok(new_b, hold_til_klub, max_teams_per_club)):
        return False

    # No duplicates → always allowed
    if len({hold_til_klub[t] for t in new_a}) == len(new_a) and \
       len({hold_til_klub[t] for t in new_b}) == len(new_b):
        return True

    # Individual outlier exception
    if is_outlier(i, old_a, distance, outlier_factor):
        return True
    if is_outlier(j, old_b, distance, outlier_factor):
        return True
    
    if driving_time_exceeds_cap(i, old_a, distance, 0.6):
        return True
    if driving_time_exceeds_cap(j, old_b, distance, 0.6):
        return True
    
    # if average_pulje_driving_time_is_over_cap(old_a, distance, 0.5):
    #     return True
    # if average_pulje_driving_time_is_over_cap(old_b, distance, 0.5):
    #     return True

    # Pulje-level outlier exception
    if pulje_is_outlier_idx(a, puljer, distance, outlier_factor):
        return True
    if pulje_is_outlier_idx(b, puljer, distance, outlier_factor):
        return True

    return False

def global_potential(puljer, distance, alpha):
    mean_term = avg_fordeling_distance(puljer, distance)
    max_term = np.var([avg_pulje_distance(p, distance) for p in puljer])
    return (1 - alpha) * mean_term + alpha * (max_term / (mean_term + 1e-9))

### Improvement algo

In [8]:
def improvement_fordeling_soft_cap(puljer_init,distance,outlier_factor=1.5,fairness_factor=0.3):
    puljer = [p.copy() for p in puljer_init]
    current_phi = global_potential(puljer, distance, fairness_factor)

    improved = True
    while improved:
        improved = False

        for a in range(len(puljer)):
            for b in range(a + 1, len(puljer)):
                old_a, old_b = puljer[a], puljer[b]

                for i in old_a:
                    for j in old_b:
                        new_a = old_a.copy()
                        new_b = old_b.copy()

                        new_a.remove(i)
                        new_b.remove(j)
                        new_a.append(j)
                        new_b.append(i)

                        if not club_constraint_ok(
                            old_a, old_b, new_a, new_b, i, j, a, b,
                            puljer, distance, hold_til_klub, outlier_factor, max_teams_per_club):
                            continue

                        puljer_tmp = puljer.copy()
                        puljer_tmp[a] = new_a
                        puljer_tmp[b] = new_b

                        new_phi = global_potential(puljer_tmp, distance, fairness_factor)

                        if new_phi < current_phi - 1e-6:
                            puljer = puljer_tmp
                            current_phi = new_phi
                            improved = True
                            break
                    if improved:
                        break
                if improved:
                    break

    return puljer

def fordel_puljer(række_df,alle_køretider_i_række,outlier_factor,fairness_factor):
    distance = make_distance_dict(alle_køretider_i_række)
    puljer_grådig = grådig_fordeling(række_df)
    return improvement_fordeling_soft_cap(puljer_grådig,distance,outlier_factor,fairness_factor)


### Visualisingsfunktioner

In [9]:
def dk_coords():
    world = gpd.read_file("dk/ne_110m_admin_0_countries.shp")
    denmark = world[world["NAME"] == "Denmark"].to_crs(epsg=4326)
    geom = denmark.geometry.iloc[0]

    # Extract existing polygons
    polygons = [p for p in geom.geoms]

    # Define rough coordinates for Fyn (Longitude, Latitude)
    fyn_coords = [
        (9.8, 55.5), 
        (10.2, 55.6),
        (10.8, 55.5),
        (10.9, 55.2),
        (10.5, 54.8),
        (9.9, 55.1), 
        (9.8, 55.5)  
    ]

    # Create the polygon and add it to the list
    fyn_polygon = Polygon(fyn_coords)
    polygons.append(fyn_polygon)
    
    dk_x = []
    dk_y = []

    for poly in polygons:
        # Get coordinates for the current island/polygon
        x, y = poly.exterior.xy
        
        # Add the coordinates to our main list
        dk_x.extend(x.tolist())
        dk_y.extend(y.tolist())
        
        # Add None to "lift the pen" so it doesn't draw a line to the next island
        dk_x.append(None)
        dk_y.append(None)
        
    return dk_x, dk_y

def plot_puljer(puljer, liga, region, division, hold_df):
    path_web = f"C:/Users/Lenovo/my-site/public/pulje_plots/{liga}_{region}_{division}"
        
    #  get dk_x and dk_y coords
    dk_x, dk_y = dk_coords()

    # 3-column setup
    fig = make_subplots(rows=1, cols=3, subplot_titles=["Hjemmebaner", "Puljefordeling", ""],
        column_widths=[0.37, 0.37, 0.26],horizontal_spacing=0.05,
        specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "table"}]])
    

    x_min, x_max = 7.5, 12.8
    y_min, y_max = 54.4, 57.9

    # Denmark background on both plots
    for col in [1, 2]:
        fig.add_trace(go.Scatter(
            x=dk_x, 
            y=dk_y, 
            mode="lines", 
            line=dict(color="rgba(120,120,120,0.35)", width=1),
            hoverinfo="skip", 
            showlegend=False
        ), row=1, col=col)

    # All teams (Plot 1)
    klubber_all = række_df["Team id"].map(hold_til_klub).values    
    customdata_all = np.column_stack([række_df["Holdnavn"].values, klubber_all])

    fig.add_trace(go.Scatter(x=række_df["Hjemmebane lng"],y=række_df["Hjemmebane lat"],
                             mode="markers",showlegend=False,marker=dict(size=8,color="rgba(80,80,80,0.6)"),
                             customdata=customdata_all,
                             hovertemplate="<b>Holdnavn:</b> %{customdata[0]}<br>""<b>Klub:</b> %{customdata[1]}<extra></extra>"),row=1,col=1)

    pulje_bogstaver = [chr(i).upper() for i in range(97, 97 + len(puljer))]
            
    # Trackers for annotations
    puljer_med_gengangere = []
    gengangere = []
    små_puljer = []
    str_af_små_puljer = []
    
    # plot pulje clustering
    for p_idx, pulje in enumerate(puljer, start=1):
        p_bogstav = pulje_bogstaver[p_idx-1]
        df = hold_df[hold_df["Team id"].isin(pulje)]
        color = f"rgba({(p_idx*25)%255},{(p_idx*60)%255},{(p_idx*110)%255},0.8)"
        klubber = df["Team id"].map(hold_til_klub).values    

        # Check for annotations logic
        vals, counts = np.unique(klubber, return_counts=True)
        klubber_med_to_hold = vals[counts>1]
        if len(klubber_med_to_hold)>0:
            gengangere.append(klubber_med_to_hold)
            puljer_med_gengangere.append(p_bogstav)
        if len(klubber) < pulje_størrelse:
            små_puljer.append(p_bogstav)
            str_af_små_puljer.append(len(pulje))

        # 1. MAP TRACE
        customdata = np.column_stack([df["Holdnavn"].values, df["Hjemmebane"].values])
        
        fig.add_trace(
            go.Scatter(
                x=df["Hjemmebane lng"], y=df["Hjemmebane lat"],
                mode="markers", name=f"Pulje {p_bogstav}",
                marker=dict(size=8, opacity=0.5, color=color),
                customdata=customdata,
                hovertemplate="<b>Hold:</b> %{customdata[0]}<br><b>Klub:</b> %{customdata[1]}<extra></extra>"), row=1, col=2)


        # 3. TABLE TRACE (Initially hidden)
        fig.add_trace(
            go.Table(
                header=dict(values=[f"<b>Holdnavn</b>", "<b>Hjemmebane</b>"],fill_color=color, align='left', font=dict(color='white')),
                cells=dict(values=[df["Holdnavn"], df["Hjemmebane"]],fill_color='white', align='left'),
                visible=False ), row=1, col=3)
        
        # 2. CENTROID TEXT
        fig.add_trace(
            go.Scatter(
                x=[df["Hjemmebane lng"].mean()], y=[df["Hjemmebane lat"].mean()],
                mode="text", text=[f"{p_bogstav}"],
                textfont=dict(color=color, size=13, family="Arial Black"),
                hoverinfo="skip", showlegend=False), row=1, col=2)


# CREATE DROPDOWN BUTTONS
    buttons = []
    show_all_vis = [True, True, True] 
    
    for _ in range(len(puljer)):
        show_all_vis.append(True)  
        show_all_vis.append(False) 
        show_all_vis.append(True) 
        
    buttons.append(dict(label="Alle puljer", method="update", args=[{"visible": show_all_vis}]))

    # Individual Pulje buttons
    for i in range(len(puljer)):
        # Start with backgrounds and Plot 1 as visible
        visibility = [True, True, True] + [False] * (len(puljer) * 3)
        
        # Calculate the starting position for this Pulje's group of 3 traces
        start_idx = 3 + (i * 3)
        
        visibility[start_idx] = True     
        visibility[start_idx + 1] = True 
        visibility[start_idx + 2] = True 
        
        buttons.append(dict(
            label=f"Pulje {pulje_bogstaver[i]}",
            method="update",
            args=[{"visible": visibility}]
        ))

    # Layout Adjustments
    fig.update_layout(
        updatemenus=[dict(
            active=0, buttons=buttons, 
            x=0.87, y=1.08,
            xanchor='center', yanchor='top'
        )],
        title={'text': f"Region {region}, {division}", 'x': 0.5, 'y': 0.98},
        title_font_weight=600,
        height=650, width=1000,
        paper_bgcolor="linen",
        showlegend=False)

    for i in [1, 2]:
        fig.update_layout({f"xaxis{i}": dict(range=[x_min, x_max], fixedrange=True),
                           f"yaxis{i}": dict(range=[y_min, y_max], fixedrange=True)})

    # Footer annotations
    klubnavne = [g[0] for g in gengangere]
    parts = [f"Pulje {p} ({k})" for p, k in zip(puljer_med_gengangere, klubnavne)]
    
    def wrap_annotation_text(text, line_width=150):
        return "<br>".join(textwrap.wrap(text, line_width))
    
    if len(parts) > 0:
        note = ("<b>Puljer med 2 hold fra samme hjemmebane:</b> "+ ", ".join(parts))
        fig.add_annotation(text=wrap_annotation_text(note),
                    xref="paper", yref="paper", font=dict(size=8),
                    x=0, y=-0.14, showarrow=False, align="left")

    if len(små_puljer) > 0:
        note = (f"<b> Puljer med færre end {pulje_størrelse} hold: </b> Pulje " + 
                ", ".join(små_puljer) + " med " + str(str_af_små_puljer[0]) + " hold")
        fig.add_annotation(text=wrap_annotation_text(note),
                    xref="paper", yref="paper", font=dict(size=8),
                    x=-0.003, y=-0.057, showarrow=False, align="left")

    fig.write_html(path_web + ".html", include_plotlyjs="cdn", full_html=True)
    return fig

In [10]:
def outline_points(polygons):
    xs, ys = [], []
    for poly in polygons:
        x, y = poly.boundary.xy
        xs.extend(x)
        ys.extend(y)
    return xs, ys

def add_denmark_background(fig, dk_x, dk_y, n_cols):
    for col in range(1, n_cols + 1):
        fig.add_trace(
            go.Scatter(
                x=dk_x,y=dk_y,mode="lines",
                line=dict(color="rgba(120,120,120,0.35)", width=1),
                hoverinfo="skip",showlegend=False),row=1, col=col)
        
def plot_puljer(puljer,liga,region,division,hold_df):
    path_web = f"C:/Users/Lenovo/my-site/public/pulje_plots/Herre_{region}_{division}"
    
    ### Plot danmark
    dk_x, dk_y = dk_coords()

    avg_distance = avg_fordeling_distance(puljer,distance)
    fig = make_subplots(rows=1,cols=2,subplot_titles=["Hjemmebaner", f"Puljefordeling"],horizontal_spacing=0.08)

    x_min, x_max = 7.5, 12.8
    y_min, y_max = 54.4, 57.9

    # Denmark background on both plots
    for col in [1, 2]:
        fig.add_trace(
            go.Scatter(x=dk_x,y=dk_y,mode="lines",line=dict(color="rgba(120,120,120,0.35)", width=1),
                hoverinfo="skip",showlegend=False),row=1,col=col)

    # Load klubber og team ids
    klubber_all = række_df["Team id"].map(hold_til_klub).values    
    customdata_all = np.column_stack([række_df["Holdnavn"].values, klubber_all])

    pulje_bogstaver = [chr(i).upper() for i in range(97, 97 + len(puljer))]
    
    # plot all teams, no pulje clustering
    fig.add_trace(
        go.Scatter(x=række_df["Hjemmebane lng"],y=række_df["Hjemmebane lat"],
            mode="markers",showlegend=False,marker=dict(size=8,color="rgba(80,80,80,0.6)"),customdata=customdata_all,
            hovertemplate="<b>Holdnavn:</b> %{customdata[0]}<br>""<b>Klub:</b> %{customdata[1]}<extra></extra>"),row=1,col=1)

    # Find puljer med gengangere og små puljer
    puljer_med_gengangere = []
    gengangere = []
    små_puljer = []
    str_af_små_puljer = []
    for p_idx, pulje in enumerate(puljer, start=1):
        p_bogstav = pulje_bogstaver[p_idx-1]
        df = hold_df[hold_df["Team id"].isin(pulje)]
        
        klubber = df["Team id"].map(hold_til_klub).values
        vals, counts = np.unique(klubber, return_counts=True)
        klubber_med_to_hold = vals[counts>1]
        
        if len(klubber_med_to_hold)>0:
                gengangere.append(klubber_med_to_hold)
                puljer_med_gengangere.append(p_bogstav)
            
        if len(klubber) < pulje_størrelse:
            små_puljer.append(p_bogstav)
            str_af_små_puljer.append(len(pulje))
            
            
    # plot pulje clustering
    for p_idx, pulje in enumerate(puljer, start=1):
        p_bogstav = pulje_bogstaver[p_idx-1]
        df = hold_df[hold_df["Team id"].isin(pulje)]
        pulje_hold_tekst = "<br>".join(f"{row.Holdnavn} – {row.Hjemmebane}" for _, row in df.iterrows())
        avg_cost = avg_pulje_distance(pulje,distance)
        max_dist = max_avg_pulje_distance(pulje,distance)

        color = f"rgba({(p_idx*25)%255},{(p_idx*60)%255},{(p_idx*110)%255},0.8)"
        lg = f"pulje_{p_bogstav}"

        klubber = df["Team id"].map(hold_til_klub).values
        customdata = np.column_stack([df["Holdnavn"].values, df["Hjemmebane"].values])
        
        fig.add_trace(
            go.Scatter(
                x=df["Hjemmebane lng"],
                y=df["Hjemmebane lat"],
                mode="markers",
                name=f"Pulje {p_bogstav}",
                legendgroup=lg,
                showlegend=True,
                marker=dict(size=10, opacity=0.6, color=color),
                customdata=customdata,
                meta = {"pulje": p_bogstav,"pulje_hold": pulje_hold_tekst},
                hovertemplate=("<b>Hold:</b> %{customdata[0]}<br>""<b>Klub:</b> %{customdata[1]}<br>""<b>Pulje %{meta.pulje}</b><br>""%{meta.pulje_hold}""<extra></extra>")),row=1,col=2)
                
                
        # Pulje annotation (centroid)
        fig.add_trace(
            go.Scatter(x=[df["Hjemmebane lng"].mean()],y=[df["Hjemmebane lat"].mean()],mode="text",
                text=[f"{p_bogstav}"],textfont=dict(color=color, size=12),
                hoverinfo="skip",showlegend=False,legendgroup=lg),row=1,col=2)

    # Layout & axis locking
    fig.update_layout(
         title={'text': f"Region {region}, {division}",'xanchor': 'center','x':0.5},
        title_font_weight = 600,
        hovermode="closest",height=650,width=900,
        legend=dict(itemclick="toggleothers",itemdoubleclick="toggle"),
        paper_bgcolor = "linen")

    for i in [1, 2]:
        fig.update_layout(
            **{
                f"xaxis{i}": dict(range=[x_min, x_max], fixedrange=True),
                f"yaxis{i}": dict(range=[y_min, y_max], fixedrange=True),})
            
            
    ### annoter plots med info om små puljer og gengangere
    klubnavne = [g[0] for g in gengangere]
    parts = [f"Pulje {p} ({k})"for p, k in zip(puljer_med_gengangere, klubnavne)]
    
    def wrap_annotation_text(text, line_width=150):
            return "<br>".join(textwrap.wrap(text, line_width))
    
    if len(parts) > 0:
        note = ("<b>Puljer med 2 hold fra samme klub:</b> "+ ", ".join(parts))
        fig.add_annotation(text=wrap_annotation_text(note),
                    xref="paper", yref="paper",font = dict(size = 8),
                    x=0, y=-0.13, showarrow=False,align="left")
        path_web = path_web + "_gengangere"

    if len(små_puljer)>0:
        små_puljer = [str(s) for s in små_puljer]
        str_af_små_puljer = [str(s) for s in str_af_små_puljer]
        note = (f"<b> Puljer med færre end {pulje_størrelse} hold: </b> Pulje "+ ", ".join(små_puljer) + " med " + ", ".join(str_af_små_puljer) + " hold")
        fig.add_annotation(text=wrap_annotation_text(note),
                    xref="paper", yref="paper",font = dict(size = 8),
                    x=0, y=-0.07, showarrow=False,align="left")
        path_web = path_web + "_lille_pulje"
    

    fig.write_html(f"plots/{liga}_{region}_{division}.html", include_plotlyjs="cdn", full_html=True)
    fig.write_html(path_web + ".html", include_plotlyjs="cdn", full_html=True)
    return fig

### Tilføj puljer som kolonne

In [11]:
def tilføj_pulje_kolonne(puljer,række_df):
    række_df.loc[:,"Pulje"] = "Ingen pulje"

    pulje_bogstaver = list(map(chr, range(97, 97 + len(puljer))))
    pulje_bogstaver = [b.capitalize() for b in pulje_bogstaver]

    for i in range(len(puljer)):
        pulje_bogstav = pulje_bogstaver[i]
        række_df.loc[række_df["Team id"].isin(puljer[i]),"Pulje"] = pulje_bogstav
        
    return række_df

## Kør program

In [12]:
# load klubber, hold og køretider
hold_df, klubber_df, køretider = forbered_data("C:/Users/Lenovo/Desktop/PadelPuljer/data/raw/Hold med hjemmebane og koordinater.xlsx")
# vælg række
herrehold = hold_df[hold_df["Liga"]=="DPF Ligaen"]
regioner = herrehold["Region"].unique()
divisioner = herrehold["Division"].unique()

rækker = pd.DataFrame()

række_df = herrehold[(herrehold["Region"]=="Øst") & (herrehold["Division"]=="Serie 4")].copy()

alle_køretider_i_række, distance = beregn_alle_køretider_i_række(række_df)
hold_til_klub = række_df.set_index("Team id")["Hjemmebane"].to_dict()
hold_til_navn = række_df.set_index("Team id")["Holdnavn"].to_dict()

# Fordel puljer
puljefordeling = fordel_puljer(række_df,alle_køretider_i_række,outlier_factor=1.2,fairness_factor=0.3)
række_df = tilføj_pulje_kolonne(puljefordeling, række_df)
plot_puljer(puljefordeling,"DPF Liga","Øst", "Serie 1", hold_df)


### Herreliga

In [ ]:
# Sørg for at alle plots i mappen slettes og dannes på ny
files = glob.glob("C:/Users/Lenovo/my-site/public/pulje_plots/*")
[os.remove(f) for f in files]

# load klubber, hold og køretider
hold_df, klubber_df, køretider = forbered_data("C:/Users/Lenovo/Desktop/PadelPuljer/data/raw/Hold med hjemmebane og koordinater.xlsx")
# vælg række
herrehold = hold_df[hold_df["Liga"]=="DPF Ligaen"]
regioner = herrehold["Region"].unique()
divisioner = herrehold["Division"].unique()

rækker = pd.DataFrame()

for region in regioner:
    for division in divisioner:
        række_df = herrehold[(herrehold["Region"]==region) & (herrehold["Division"]==division)].copy()

        if len(række_df) == 0:
            continue
        
        print(f"Beregner puljefordeling for region {region}, division {division}...")

        alle_køretider_i_række, distance = beregn_alle_køretider_i_række(række_df)
        hold_til_klub = række_df.set_index("Team id")["Hjemmebane"].to_dict()
        hold_til_navn = række_df.set_index("Team id")["Holdnavn"].to_dict()

        # Fordel puljer
        puljefordeling = fordel_puljer(række_df,alle_køretider_i_række,outlier_factor=1.2,fairness_factor=0.3)
        række_df = tilføj_pulje_kolonne(puljefordeling, række_df)
        plot_puljer(puljefordeling,"DPF Liga",region, division, hold_df)
        rækker = pd.concat([rækker,række_df])

rækker.to_excel(f"C:/Users/Lenovo/my-site/public/rækker.xlsx")

Beregner puljefordeling for region Vest, division Serie 1...


### Alle ligaer

In [ ]:
# Sørg for at alle plots i mappen slettes og dannes på ny
# files = glob.glob("C:/Users/Lenovo/my-site/public/pulje_plots/*")
# [os.remove(f) for f in files]

# load klubber, hold og køretider
hold_df, klubber_df, køretider = forbered_data("C:/Users/Lenovo/Desktop/PadelPuljer/data/raw/Hold med hjemmebane og koordinater.xlsx")
hold_til_klub = hold_df.set_index("Team id")["Hjemmebane"].to_dict()
hold_til_navn = hold_df.set_index("Team id")["Holdnavn"].to_dict()

# vælg række
ligaer = hold_df["Liga"].unique()
regioner = hold_df["Region"].unique()
divisioner = hold_df["Division"].unique()

rækker = pd.DataFrame()


for liga in ligaer:
    print(f"Beregner puljefordelinger for {liga}...")
    liga = hold_df[hold_df["Liga"]==liga]
        
    for region in regioner:
        for division in divisioner:
            
            række_df = liga[(liga["Region"]==region) & (liga["Division"]==division)].copy()
            if len(række_df) == 0:
                continue
            
            print(len(række_df))
            
            print(f"Beregner puljefordeling for region {region}, division {division}...")

            alle_køretider_i_række, distance = beregn_alle_køretider_i_række(række_df)

            # Fordel puljer
            puljefordeling = fordel_puljer(række_df,alle_køretider_i_række,outlier_factor=1.5,fairness_factor=0.4)
            række_df = tilføj_pulje_kolonne(puljefordeling, række_df)
            plot_puljer(puljefordeling, liga, region, division, hold_df)
            rækker = pd.concat([rækker,række_df])
            
rækker.to_excel(f"C:/Users/Lenovo/my-site/public/rækker.xlsx")


100%|██████████| 180/180 [00:29<00:00,  6.10it/s]


Beregner puljefordelinger for DPF Ligaen 4P...
7
Beregner puljefordeling for region 4. Sønderjylland, division 2. Division...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Lenovo\\my-site\\public\\pulje_plots\\      Team id           Liga              Region       Division  Division ny  \\\n0     2703845  DPF Ligaen 4P    4. Sønderjylland    2. Division          NaN   \n1     2703929  DPF Ligaen 4P  5. Midtjylland Øst    2. Division          NaN   \n2     2703936  DPF Ligaen 4P  5. Midtjylland Øst        Serie 2          NaN   \n3     2703842  DPF Ligaen 4P  5. Midtjylland Øst    1. Division          NaN   \n4     2703720  DPF Ligaen 4P  5. Midtjylland Øst  Danmarksserie          NaN   \n...       ...            ...                 ...            ...          ...   \n1732  2771183  DPF Ligaen 4P            Nye hold       Nye hold          NaN   \n1733  2770746  DPF Ligaen 4P            Nye hold       Nye hold          NaN   \n1734  2770643  DPF Ligaen 4P            Nye hold       Nye hold          NaN   \n1735  2770683  DPF Ligaen 4P            Nye hold       Nye hold          NaN   \n17

: 

In [ ]:
def max_hold_samme_klub(pulje):
    """ Returnerer antal gange den mest repræsenterede klub er repræsenteret i en pulje"""
    klubber = [hold_til_klub[h] for h in pulje]
    _, counts = np.unique(klubber, return_counts=True)
    return counts.max()

def metrics_for_fordeling(puljer):
    """ Returnerer en række metrikker for en puljefordeling"""
    avg_pulje = avg_fordeling_distance(puljer,distance)
    max_hold = max_fordeling_distance(puljer,distance)
    avg_pr_pulje = [avg_pulje_distance(p,distance) for p in puljer]
    max_same_club = [max_hold_samme_klub(p) for p in puljer]
    
    return {
        "Antal puljer": len(puljer),
        "Gns. køretid (pulje-gennemsnit)": avg_pulje,
        "Min gns. køretid (bedste pulje)": min(avg_pr_pulje),
        "Max gns. køretid (værste pulje)": max(avg_pr_pulje),
        "Maks. køretid (enkelt hold)": max_hold,
        "Puljer med 2 hold fra samme klub": sum(m == 2 for m in max_same_club),
        "Puljer med 3 hold fra samme klub": sum(m == 3 for m in max_same_club),
    }

df_metrics = pd.DataFrame({
    "Outlier factor 1.6": metrics_for_fordeling(puljefordeling)
})
df_metrics

,Outlier factor 1.6
Antal puljer,9.000000
Gns. køretid (pulje-gennemsnit),1.828695
Min gns. køretid (bedste pulje),1.017889
Max gns. køretid (værste pulje),2.952800
Maks. køretid (enkelt hold),3.718991
Puljer med 2 hold fra samme klub,4.000000
Puljer med 3 hold fra samme klub,0.000000


In [ ]:
# def outline_points(polygons):
#     xs, ys = [], []
#     for poly in polygons:
#         x, y = poly.boundary.xy
#         xs.extend(x)
#         ys.extend(y)
#     return xs, ys

# def add_denmark_background(fig, dk_x, dk_y, n_cols):
#     for col in range(1, n_cols + 1):
#         fig.add_trace(
#             go.Scatter(
#                 x=dk_x,y=dk_y,mode="lines",
#                 line=dict(color="rgba(120,120,120,0.35)", width=1),
#                 hoverinfo="skip",showlegend=False),row=1, col=col)

# def plot_puljer(puljer,liga, region,division,hold_df):
#     path_web = f"C:/Users/Lenovo/my-site/public/pulje_plots/{liga}_{region}_{division}"
    
#     ### Plot danmark
#     world = gpd.read_file("dk/ne_110m_admin_0_countries.shp")
    
#     denmark = world[world["NAME"] == "Denmark"].to_crs(epsg=4326)
#     geom = denmark.geometry.iloc[0]
#     polygons = [p for p in geom.geoms]

#     dk_x, dk_y = outline_points(polygons)
#     avg_distance = avg_fordeling_distance(puljer,distance)
#     fig = make_subplots(rows=1,cols=2,subplot_titles=["Hjemmebaner", f"Puljefordeling"],horizontal_spacing=0.08)

#     x_min, x_max = 7.5, 12.8
#     y_min, y_max = 54.4, 57.9

#     # Denmark background on both plots
#     for col in [1, 2]:
#         fig.add_trace(
#             go.Scatter(x=dk_x,y=dk_y,mode="lines",line=dict(color="rgba(120,120,120,0.35)", width=1),
#                 hoverinfo="skip",showlegend=False),row=1,col=col)

#     # Load klubber og team ids
#     klubber = række_df["Team id"].map(hold_til_klub).values    
#     customdata = np.column_stack([række_df["Holdnavn"].values,klubber])


#     # plot all teams, no pulje clustering
#     fig.add_trace(
#         go.Scatter(x=række_df["Hjemmebane lng"],y=række_df["Hjemmebane lat"],
#             mode="markers",showlegend=False,marker=dict(size=8,color="rgba(80,80,80,0.6)"),
#             customdata=customdata,
#                 hovertemplate="<b>Holdnavn:</b> %{customdata[0]}<br>""<b>Klub:</b> %{customdata[1]}<extra></extra>"
#             ),row=1,col=1)

    
#     pulje_bogstaver = list(map(chr, range(97, 97 + len(puljer))))
#     pulje_bogstaver = [b.capitalize() for b in pulje_bogstaver]
            
#     # Find puljer med gengangere og små puljer
#     puljer_med_gengangere = []
#     gengangere = []
#     små_puljer = []
#     str_af_små_puljer = []
#     for p_idx, pulje in enumerate(puljer, start=1):
        
#         p_bogstav = pulje_bogstaver[p_idx-1]
#         df = hold_df[hold_df["Team id"].isin(pulje)]
        
#         klubber = df["Team id"].map(hold_til_klub).values
#         vals, counts = np.unique(klubber, return_counts=True)
#         klubber_med_to_hold = vals[counts>1]
        
#         if len(klubber_med_to_hold)>0:
#                 gengangere.append(klubber_med_to_hold)
#                 puljer_med_gengangere.append(p_bogstav)
            
#         if len(klubber) < pulje_størrelse:
#             små_puljer.append(p_bogstav)
#             str_af_små_puljer.append(len(pulje))
            
            
#     # plot pulje clustering
#     for p_idx, pulje in enumerate(puljer, start=1):
#         p_bogstav = pulje_bogstaver[p_idx-1]
#         df = hold_df[hold_df["Team id"].isin(pulje)]
#         color = f"rgba({(p_idx*25)%255},{(p_idx*60)%255},{(p_idx*110)%255},0.8)"
#         lg = f"pulje_{p_bogstav}"


#         klubber = række_df["Team id"].map(hold_til_klub).values    
#         customdata = np.column_stack([række_df["Holdnavn"].values,klubber])
#         fig.add_trace(
#             go.Scatter(
#                 x=df["Hjemmebane lng"],
#                 y=df["Hjemmebane lat"],
#                 mode="markers",
#                 name=f"Pulje {p_bogstav}",
#                 legendgroup=lg,
#                 showlegend=False,
#                 marker=dict(size=8, opacity=0.5, color=color),
#                 customdata=customdata,
#                 text=p_bogstav,
#                 hovertemplate=
#                     "<b>Pulje:</b> %{fullData.name}<br>"
#                     "<b>Holdnavn:</b> %{customdata[0]}<br>"
#                     "<b>Klub:</b> %{customdata[1]}"
#                     "<extra></extra>"),row=1,col=2)
        
#         # Pulje annotation (centroid)
#         fig.add_trace(
#             go.Scatter(x=[df["Hjemmebane lng"].mean()],y=[df["Hjemmebane lat"].mean()],mode="text",
#                 text=[f"{p_bogstav}"],textfont=dict(color=color, size=12),
#                 hoverinfo="skip",showlegend=False,legendgroup=lg),row=1,col=2)
        
#         fig.add_trace(
#             go.Scatter(
#                 x=df["Hjemmebane lng"],
#                 y=df["Hjemmebane lat"],
#                 mode="markers",
#                 name=f"Pulje {p_idx}",
#                 marker=dict(size=10, opacity=0.4),
#                 legendgroup=f"pulje_{p_idx}",
#                 showlegend=False))

#     # Layout & axis locking
#     fig.update_layout(
#          title={'text': f"Region {region}, {division}",'xanchor': 'center','x':0.5},
#         title_font_weight = 600,
#         hovermode="closest",height=700,width=1000,
#         legend=dict(itemclick="toggleothers",itemdoubleclick="toggle"),
#         paper_bgcolor = "linen")

#     for i in [1, 2]:
#         fig.update_layout(
#             **{
#                 f"xaxis{i}": dict(range=[x_min, x_max], fixedrange=True),
#                 f"yaxis{i}": dict(range=[y_min, y_max], fixedrange=True),})
            
            
#     ### annoter plots med info om små puljer og gengangere
#     klubnavne = [g[0] for g in gengangere]
#     parts = [f"Pulje {p} ({k})"for p, k in zip(puljer_med_gengangere, klubnavne)]
    
#     def wrap_annotation_text(text, line_width=150):
#             return "<br>".join(textwrap.wrap(text, line_width))
    
#     if len(parts) > 0:
#         note = ("<b>Puljer med 2 hold fra samme hjemmebane:</b> "+ ", ".join(parts))
#         fig.add_annotation(text=wrap_annotation_text(note),
#                     xref="paper", yref="paper",font = dict(size = 8),
#                     x=0, y=-0.13, showarrow=False,align="left")
#         path_web = path_web + "_gengangere"

#     if len(små_puljer)>0:
#         små_puljer = [str(s) for s in små_puljer]
#         str_af_små_puljer = [str(s) for s in str_af_små_puljer]
#         note = (f"<b> Puljer med færre end {pulje_størrelse} hold: </b> Pulje "+ ", ".join(små_puljer) + " med " + ", ".join(str_af_små_puljer) + " hold")
#         fig.add_annotation(text=wrap_annotation_text(note),
#                     xref="paper", yref="paper",font = dict(size = 8),
#                     x=0, y=-0.07, showarrow=False,align="left")
#         path_web = path_web + "_lille_pulje"
    

#     fig.write_html(f"plots/{liga}_{region}_{division}.html", include_plotlyjs="cdn", full_html=True)
#     fig.write_html(path_web + ".html", include_plotlyjs="cdn", full_html=True)
#     return fig

In [ ]:
# def saml_små_puljer(puljer):
#     """ En række restpuljer kan opstå for de sidste puljer i den grådige algoritme
#     Disse småpuljer samles i første omgang til puljer af præcis 7 hold og herefter en evt. rest.
#     En evt rest fordeles i funktionen fordel_små_puljer()"""
#     gyldige_puljer = []
#     restpulje = []

#     # Separate valid puljer and collect leftovers
#     for pulje in puljer:
#         if len(pulje) < pulje_størrelse:
#             restpulje.extend(pulje)
#         else:
#             gyldige_puljer.append(pulje)

#     # Split restpulje into chunks of size pulje_størrelse
#     for i in range(0, len(restpulje), pulje_størrelse):
#         gyldige_puljer.append(restpulje[i:i + pulje_størrelse])

#     return gyldige_puljer

# def håndhæv_max_to_klubber(puljer):
#     """ Tjekker om nogen puljer indeholder mere end 2 hold fra samme klub
#     I så fald foretages et tilfældigt bytte med et hold fra en anden pulje
#     Byttet foretages under hensyn til ikke hermed at overstige max kravet"""
#     for pulje in puljer:
#         klubber_i_pulje = np.array([hold_til_klub[h] for h in pulje])
#         vals, counts = np.unique(klubber_i_pulje, return_counts=True)
#         overrepræsenterede_klubber = vals[counts > max_teams_per_club]
#         overrepræsenterede_hold = [h for h in pulje if hold_til_klub[h] in overrepræsenterede_klubber]
#         if overrepræsenterede_hold:
#             for hold_ind in overrepræsenterede_hold[:]:

#                 for _, pulje_at_swappe in enumerate(puljer):
#                     if pulje_at_swappe == pulje:
#                         continue
#                     klubber_i_swap = [hold_til_klub[h] for h in pulje_at_swappe]
#                     if klubber_i_swap.count(hold_til_klub[hold_ind]) >= 2:
#                         continue
#                     hold_ud = pulje_at_swappe[0]
                    
#                     pulje_at_swappe.remove(hold_ud)
#                     pulje.remove(hold_ind)
                    
#                     pulje.append(hold_ud)
#                     pulje_at_swappe.append(hold_ind)

#                     break 
#     return puljer


# def fordel_små_puljer(puljer):
#     """Hvis antal_hold \\ antal_hold_pr_pulje ikke går op med heltalsdivision fordeles de tiloversblevne hold
#     Hvis der er en rest på mindre end min_pulje_størrelse fordeles de resterende hold, således at puljer
#     med pulje_størrelse + 1 dannes."""
#     for pulje in puljer:
#         if len(pulje) < min_pulje_størrelse:
#             for hold_ind in pulje[:]:
#                 for _, pulje_at_swappe in enumerate(puljer):
#                     if pulje_at_swappe == pulje:
#                         continue
                    
#                     pulje.remove(hold_ind)
#                     pulje_at_swappe.append(hold_ind)

#                     break 
#             puljer.remove(pulje)
#     return puljer
            

# def grådig_fordeling(række_df):    
#     """ Danner første bud på puljer gennem en grådig fordelingsalgoritme
#     Gennemgår hold og vælger grådigt de 6 nærmeste hold, der ikke er fra samme klub
    
#     For de sidst fordelte puljer vil der sjældent være 6 hold fra andre klubber tilbage.
#     Her brydes puljerne op i puljer med mindre end 7 hold i hver pulje.
#     Disse samles, hvorefter eventuelle overtrædelser af max_to_hold_fra_samme_klub-reglen håndteres ved tilfældige byt
    
#     """ 
#     hold_liste = række_df["Team id"].tolist()
#     unused_teams = set(hold_liste)
#     puljer_grådig = []

#     while unused_teams:
#         team = unused_teams.pop() 
#         klub = hold_til_klub[team]

#         possible_neighbors = alle_køretider_i_række[(alle_køretider_i_række["Team id hjemmebane"]==team) & \
#                                 (alle_køretider_i_række["Team id udebane"].isin(unused_teams)) & \
#                                 (alle_køretider_i_række["Udebane"] != klub)]

#         possible_neighbors = possible_neighbors.drop_duplicates(subset="Udebane",keep="first")
#         nearest_neighbors = possible_neighbors.sort_values(by="Køretid").head(pulje_størrelse - 1)
#         nearest_neighbors = nearest_neighbors["Team id udebane"].tolist()
#         pulje = [team] + nearest_neighbors
#         puljer_grådig.append(pulje)
        
#         for team in nearest_neighbors:
#             unused_teams.remove(team)
            
#     puljer_grådig = saml_små_puljer(puljer_grådig)
#     puljer_grådig = håndhæv_max_to_klubber(puljer_grådig)
#     puljer_grådig = fordel_små_puljer(puljer_grådig)
#     puljer_grådig = håndhæv_max_to_klubber(puljer_grådig)
    
    
#     return puljer_grådig
